<a href="https://colab.research.google.com/github/muhuuh/rl_educ/blob/main/pytorch_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install torch torchvision torchaudio  # Core PyTorch + vision/audio helpers

In [1]:
import torch
print(torch.__version__)  # Should print 2.4.0 or similar
print("CUDA available?" if torch.cuda.is_available() else "Using CPU")  # GPU check (fine on CPU for now)

2.9.0+cu126
CUDA available?


In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26.4M/26.4M [00:01<00:00, 19.0MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 302kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 5.61MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 10.9MB/s]


In [7]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [8]:
# Detect device (use GPU if available; your AMD Radeon 890M can work with ROCm)
device = "cuda" if torch.cuda.is_available() else "cpu"  # For AMD, it might be "rocm" if set up

# Define the model class
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()  # Initialize the parent class
        self.flatten = nn.Flatten()  # Flatten 28x28 image to 1D vector (784 elements)
        self.linear_relu_stack = nn.Sequential(  # Stack layers sequentially
            nn.Linear(28*28, 512),  # Input: 784, Output: 512 (fully connected layer)
            nn.ReLU(),              # Activation: Introduces non-linearity
            nn.Linear(512, 512),    # Hidden layer
            nn.ReLU(),
            nn.Linear(512, 10)      # Output: 10 classes (logits, not probabilities yet)
        )

    def forward(self, x):  # Defines how data flows through the model
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# Instantiate and move to device
model = NeuralNetwork().to(device)
print(model)  # Prints the model architecture

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [13]:
loss_fn = nn.CrossEntropyLoss()  # Combines softmax and negative log-likelihood for classification
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)  # Stochastic Gradient Descent; lr = learning rate

In [14]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()  # Training mode (enables dropout if any)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)  # Forward pass
        loss = loss_fn(pred, y)  # Compute loss
        loss.backward()  # Backprop: Compute gradients
        optimizer.step()  # Update parameters
        optimizer.zero_grad()  # Reset gradients
        if batch % 100 == 0:
            print(f"Loss: {loss.item():>7f} [{batch * len(X):>5d}/{size:>5d}]")

In [17]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()  # Evaluation mode
    test_loss, correct = 0, 0
    with torch.no_grad():  # Disable gradients for efficiency
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Accuracy: {100*correct:>0.1f}%, Avg loss: {test_loss:>8f}")

In [16]:
epochs = 5  # Number of full passes over the dataset
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
Loss: 1.145434 [    0/60000]
Loss: 1.053073 [ 6400/60000]
Loss: 0.793041 [12800/60000]
Loss: 0.915642 [19200/60000]
Loss: 0.768665 [25600/60000]
Loss: 0.742936 [32000/60000]
Loss: 0.806751 [38400/60000]
Loss: 0.736057 [44800/60000]
Loss: 0.756030 [51200/60000]
Loss: 0.708990 [57600/60000]
Accuracy: 75.3%, Avg loss: 0.697548
Epoch 2
-------------------------------
Loss: 0.659444 [    0/60000]
Loss: 0.735792 [ 6400/60000]
Loss: 0.494879 [12800/60000]
Loss: 0.718527 [19200/60000]
Loss: 0.625253 [25600/60000]
Loss: 0.592931 [32000/60000]
Loss: 0.637309 [38400/60000]
Loss: 0.644374 [44800/60000]
Loss: 0.673804 [51200/60000]
Loss: 0.593441 [57600/60000]
Accuracy: 79.4%, Avg loss: 0.595869
Epoch 3
-------------------------------
Loss: 0.515943 [    0/60000]
Loss: 0.606982 [ 6400/60000]
Loss: 0.404304 [12800/60000]
Loss: 0.632514 [19200/60000]
Loss: 0.563267 [25600/60000]
Loss: 0.543369 [32000/60000]
Loss: 0.556666 [38400/60000]
Loss: 0.633530 [44800/600